# ResNet18 and ResNet50

In [1]:
from utils import *
import torch
import torch.nn as nn
from tqdm import tqdm
from torchvision import models
import wandb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 100
lr = 1e-3
batch_size = 8
model_size = 50

wandb.init(project="Digital Medicine Case 2", entity="tommytyc")
wandb.config = {
    "learning_rate": lr,
    "epochs": epochs,
    "batch_size": batch_size,
    "model_size": model_size,
}

train_df = pd.read_csv("./Data/train_label.csv")
valid_df = pd.read_csv("./Data/valid_label.csv")
train_set = Covid_Dataset(train_df, 'train')
train_loader = DataLoader(train_set, batch_size=batch_size)
valid_set = Covid_Dataset(valid_df, 'valid')
valid_loader = DataLoader(valid_set, batch_size=1)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tommytyc (use `wandb login --relogin` to force relogin)


## Model

In [2]:
class ResNet(nn.Module):
    def __init__(self, model_size=18, pretrained=True):
        super(ResNet, self).__init__()
        if model_size == 18:
            last_dim = 512
        elif model_size == 50:
            last_dim = 2048
        
        self.model_size = model_size
        pretrained_model = models.__dict__[f"resnet{model_size}"](pretrained=pretrained)
        
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = pretrained_model._modules["bn1"]
        self.relu = pretrained_model._modules["relu"]
        self.maxpool = pretrained_model._modules["maxpool"]
        
        self.layer1 = pretrained_model._modules["layer1"]
        self.layer2 = pretrained_model._modules["layer2"]
        self.layer3 = pretrained_model._modules["layer3"]
        self.layer4 = pretrained_model._modules["layer4"]
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(last_dim, 3)
        
        del pretrained_model

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

## Training

In [3]:
model = ResNet(model_size=model_size, pretrained=True)
ce_loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=1e-4)
model.to(device)
ce_loss.to(device)
for epoch in tqdm(range(epochs)):
    model.train()
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = ce_loss(outputs, labels)
        loss.backward()
        optimizer.step()
    wandb.log({"loss": loss})
    if epoch % 5 == 0:
        print(f"Epoch: {epoch}, loss: {loss}")

  1%|          | 1/100 [01:21<2:14:33, 81.55s/it]

Epoch: 0, loss: 0.899376392364502


  6%|▌         | 6/100 [08:09<2:07:51, 81.61s/it]

Epoch: 5, loss: 0.48309552669525146


 11%|█         | 11/100 [14:58<2:01:13, 81.73s/it]

...

 91%|█████████ | 91/100 [2:04:04<12:16, 81.87s/it]

Epoch: 90, loss: 0.014825035817921162


 96%|█████████▌| 96/100 [2:10:53<05:27, 81.87s/it]

Epoch: 95, loss: 0.0009966291254386306


100%|██████████| 100/100 [2:16:21<00:00, 81.81s/it]


## Predicting

In [4]:
torch.save(model.state_dict(), f"./Models/resnet{model_size}_covid_model.pt")

In [5]:
cat_transform = {
    0: "Atypical",
    1: "Negative", 
    2: "Typical"
}

model = ResNet(model_size=model_size, pretrained=True)
model.load_state_dict(torch.load(f"./Models/resnet{model_size}_covid_model.pt"))
model.to(device)
model.eval()
out_df = pd.DataFrame(columns=["FileID", "Type"])
cnt = 0
with torch.no_grad():
    for images, filename in valid_loader:
        images = images.float().to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        out_df.loc[cnt] = [filename[0], cat_transform[preds.item()]]
        cnt += 1

In [6]:
out_df.sort_values(by="FileID", inplace=True)
out_df.to_csv(f"./Results/resnet{model_size}_covid_model.csv", index=False)